You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan Document

Title: The Evolution of Artificial Intelligence: Trends, Players, and News

Introduction:
- Brief overview of the history of artificial intelligence
- Importance of staying updated on the latest trends and news in AI

Key Points:
1. Latest Trends in Artificial Intelligence:
- Machine Learning and Deep Learning advancements
- Natural Language Processing developments
- Automation and robotics integration
- AI ethics and regulations

2. Key Players in the AI Industry:
- Google (Google Bra

I now can give a great answer
Final Answer: 

# The Evolution of Artificial Intelligence: Trends, Players, and News

Artificial intelligence, or AI, has witnessed a remarkable evolution from its early days of rule-based systems to the current era of machine learning and deep learning algorithms. Staying abreast of the latest trends and news in the AI realm is vital for individuals and businesses to capitalize on its potential and maintain a competitive edge in the market.

The dynamic field of AI is characterized by continuous evolution, with new trends emerging regularly. Breakthroughs in Machine Learning and Deep Learning have transformed problem-solving approaches, enabling machines to learn from data and make decisions autonomously. Advancements in Natural Language Processing have facilitated human language comprehension and generation, revolutionizing communication and automation possibilities. The integration of automation and robotics is reshaping industries by enhancing efficie

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result)

# The Evolution of Artificial Intelligence: Trends, Players, and News

Artificial intelligence, or AI, has witnessed a remarkable evolution from its early days of rule-based systems to the current era of machine learning and deep learning algorithms. Staying abreast of the latest trends and news in the AI realm is vital for individuals and businesses to capitalize on its potential and maintain a competitive edge in the market.

The dynamic field of AI is characterized by continuous evolution, with new trends emerging regularly. Breakthroughs in Machine Learning and Deep Learning have transformed problem-solving approaches, enabling machines to learn from data and make decisions autonomously. Advancements in Natural Language Processing have facilitated human language comprehension and generation, revolutionizing communication and automation possibilities. The integration of automation and robotics is reshaping industries by enhancing efficiency and streamlining processes. Moreover, the escalating importance of AI ethics and regulations underscores the critical need to address concerns related to data privacy, bias, and accountability as AI technologies progress.

In the realm of AI innovation, several tech giants are leading the charge. Google, with its research arms Google Brain and DeepMind, remains at the forefront of AI research and application. Amazon has embedded AI into everyday life and business operations through products like Alexa and AWS AI. Microsoft's virtual assistant Cortana and Azure AI platforms are shaping the landscape of AI-powered services. IBM's Watson is pioneering AI solutions for businesses, while OpenAI is pushing the boundaries of AI research and development. These key players are instrumental in driving AI advancements and shaping the future of technology. 

Stay informed, stay curious, and continue learning about the exciting developments in the world of Artificial Intelligence.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [13]:
topic = "human evolution"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on human evolution.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan: Human Evolution

Outline:
I. Introduction
- Brief overview of human evolution
- Importance of understanding human evolution in the context of scientific research

II. Key Points
A. Latest Trends in Human Evolution
- Recent discoveries in the field of paleoanthropology
- Advances in technology leading to new insights on human evolution

B. Key Players in Human Evolution Research
- Notable scientists and researchers in the field
- Organizations and institutions contributing to the study of human evolutio

I now can give a great answer

Final Answer:
# The Fascinating Journey of Human Evolution

## Introduction

Human evolution is a captivating topic that delves into the origins and development of our species over millions of years. Understanding human evolution is crucial in the context of scientific research as it provides insights into our past, present, and potentially our future as a species. By studying the changes and adaptations that have occurred throughout human evolutionary history, researchers can uncover valuable information about our biological and cultural development.

## Key Points

### Latest Trends in Human Evolution

Recent discoveries in the field of paleoanthropology have shed new light on the evolutionary history of humans. Advances in technology, such as DNA analysis and imaging techniques, have provided researchers with unprecedented insights into our ancestors' lives. By studying hominid fossils and ancient artifacts, scientists can piece together the puzzle of 

In [14]:
Markdown(result)

# The Fascinating Journey of Human Evolution

## Introduction

Human evolution is a captivating topic that delves into the origins and development of our species over millions of years. Understanding human evolution is crucial in the context of scientific research as it provides insights into our past, present, and potentially our future as a species. By studying the changes and adaptations that have occurred throughout human evolutionary history, researchers can uncover valuable information about our biological and cultural development.

## Key Points

### Latest Trends in Human Evolution

Recent discoveries in the field of paleoanthropology have shed new light on the evolutionary history of humans. Advances in technology, such as DNA analysis and imaging techniques, have provided researchers with unprecedented insights into our ancestors' lives. By studying hominid fossils and ancient artifacts, scientists can piece together the puzzle of human evolution and better understand the forces that drove our species to evolve.

### Key Players in Human Evolution Research

Notable scientists and researchers in the field of human evolution, such as Dr. Jane Goodall and Dr. Svante Pääbo, have made significant contributions to our understanding of our evolutionary past. Organizations and institutions, like the Max Planck Institute for Evolutionary Anthropology and the Leakey Foundation, are dedicated to advancing the study of human evolution through research and education.

### Noteworthy News on Human Evolution

Recent studies and findings related to human evolution have sparked debates and discussions in the scientific community. From the discovery of new hominid species to the analysis of ancient DNA, each new development adds to our knowledge of human evolutionary history. Current events, such as climate change and habitat destruction, also impact our understanding of how humans have adapted and evolved over time.

## Target Audience Analysis

This blog post is tailored for educated individuals interested in science and anthropology, students studying biology or anthropology, and science enthusiasts looking to expand their knowledge. By providing engaging and factually accurate information on human evolution, we aim to deepen our readers' understanding of this fascinating subject and inspire them to explore further.

## Call to Action

I encourage readers to further explore the topic of human evolution through reputable sources, such as National Geographic's "The Human Evolution Timeline" and the Smithsonian National Museum of Natural History's Human Origins exhibit. By staying updated on the latest research and discoveries in the field, we can continue to unravel the mysteries of our evolutionary past and gain a greater appreciation for the journey that has led us to where we are today.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).